# TopoSPAM - Topological Simulation Platform for Active Matter
## Active 3D vertex model example


In [10]:
import topospam as topospam
%matplotlib inline
repo_path=topospam.set_repo_path('..')

Success: The path '..' contains the TopoSPAM repository.


## Active 3D vertex model on a sphere

We build an active 3D vertex model to simulate the dynamics of a biological tissue which actively interacts with its environment. 

In general, in a Vertex model, space is entirely subdivided into cells, which are each represented by a polygon (or polyhedron). For the cells, parameters like a preferred cell surface area or circumference can be defined. Each cell's deviations from these preferred values contributes to the so-called work function $W$ of the Vertex model

$$ W = \sum_{\alpha \in \text{cells}} \frac{1}{2} K^{\alpha} \left( A^{\alpha} - A_{0}^{\alpha} \right)^2 + \sum_{\alpha \in \text{cells}} \frac{1}{2} \Lambda^{\alpha} L^{\alpha} .$$

Here, $A^{\alpha}$ is the surface are a of cell $\alpha$, $A_{0}^{\alpha}$ its respective preferred value and $K^{\alpha}$ the cell's area stiffness. Furthermore, $L^{\alpha}$ is the cell's perimeter and $\Lambda^{\alpha}$ its bond tension.  

In a standard vertext model, the dynamics of the system is entirely determined by the work function. In each step, the model attempts to move the vertices (i.e. the 3-cell contact points) in such a way as to minimise $W$. 

In contrast, we here provide an active vertex model, meaning that the cells additionally exert active forces on their surroundings and thus influence the dynamics of the system. Thus, at each vertex $m$ located at position $\mathbf{X}_m$, force balance reads

$$ \xi \mathbf{v}_m = F \langle \mathbf{p} \rangle_m - \frac{\partial W}{\partial \mathbf{X}_m} + f_m^n \mathbf{\hat{n}}_m ,$$

where $\mathbf{m}_m$ is the velocity of vertex m, and $\xi$ is the friction coefficient with the environment. The first term on the right hand side describes traction forces of magnitude $F$ exerted by the (three) cells abutting at vertex $m$ and in the average direction of their polarities $\langle \mathbf{p} \rangle_m = \sum_{\langle \alpha | m \rangle } \mathbf{p}_{\alpha} / M_{\alpha}$, where $\mathbf{p}_{\alpha}$ and $M_{\alpha}$ are each cell's polarity and number of vertices, respectively. $f_m^n$ is the magnitude of the normal force at vertex $m$.
Note that we here built a model with cells on a sphere where each cell is represented by a polygon on the sphere surface and that we consider a non-deforming geometry by setting $\mathbf{v}_m \cdot \mathbf{\hat{n}}_m = 0$.

In our model, the dynamics of the cell polarity vectors, which direct the traction forces exerted by the cells, is given by

$$ \frac{D \mathbf{p}_{\alpha}}{D t} = \gamma \langle \mathbf{p} \rangle_{\alpha} + \sqrt{2 \mathcal{D}_r} \mathbf{\eta}_{t} + \mu \mathbf{p}_{\alpha} + p_{\alpha}^n \mathbf{\hat{n}}_{\alpha} .$$

Here, $D/D t$ denotes a co-rotational time derivative. The first term on the right hand side accounts for the alignment of cell polarity with the average polarity of its $M_{\alpha}$ neares neighbors $ \langle p \rangle_{\alpha} = (1 / M_{\alpha}) \sum_{\langle \alpha' | \alpha \rangle} \mathbf{p}_{\alpha'}$ with the rate $\gamma$. The second term accounts for rotational noise with a diffusion coefficient $\mathcal{D}_r$. The polarity noise $\mathbf{\eta}(t) = \mathbf{\hat{s}}_{\perp} \eta(t)$ is perpendicular to both cell polarity and the normal vector at the cell center, $\mathbf{\hat{s}}_{\perp} = \mathbf{\hat{n}}_{\alpha} \times \mathbf{p}_{\alpha} / \left| \mathbf{\hat{n}}_{\alpha} \times \mathbf{p}_{\alpha} \right|$. The polarity noise magnitude $\eta$ is a Gaussian variable with mean 0 and variance 1. The third term is used to impose $ \left| \mathbf{p}_{\alpha} \right| = 1$ at each time through a Lagrange multiplier $\mu(t) = - \gamma \mathbf{p}_{\alpha} \cdot \langle \mathbf{p} \rangle_{\alpha}$. the last term ensures that the polarity remains in the tangent plane of the sphere by adding a normal component with the magnitude $p_{\alpha}^n = - \gamma \langle \mathbf{p} \rangle_{\alpha} \cdot \mathbf{\hat{n}}_{\alpha}$.

Further details as well as an application of this model can be found in the literature: 

(https://www.biorxiv.org/content/10.1101/2022.09.29.510101v1)



!!!!!!!!!!!!!!!The main source code with intiial conditions is located in bin/Active2d.cpp. This file can be modified as per user needs for different geometries and initial conditions. 

In [11]:
VertexModelSimulator = topospam.vertex_model(repo_path)

In [12]:
VertexModelSimulator.params.time_step    = 0.2
# ...number of time steps after which is frame will be written to output
VertexModelSimulator.params.ninfo    = 10
# ...the simulation will continue for nframes frames.
# ...therefore, total steps = nframes * ninfo
VertexModelSimulator.params.nframes    = 100
# ...if set, after this many frames the noise on polarity turns off
# ...if commented out, it gets default value: std::numeric_limits<unsigned>::max()
VertexModelSimulator.params.noiseoff    = 2800
# ...by default vertices are constrained on a spherical surface.
# ...to relax this constrain and allow for radial deformations, un-comment:
#VertexModelSimulator.params.spherical_constrain       = 0

# ==============================  model parameters     ===================================
# ...number of cells
VertexModelSimulator.params.Nc       = 200
# ...cell area stiffness
VertexModelSimulator.params.Kc       = 1.0
# ...cell preferred area
VertexModelSimulator.params.A0c       = 1.0
# ...cell bond tension
VertexModelSimulator.params.bond_Tension   = 0.1
# ...cell perimeter elasticity
VertexModelSimulator.params.perim_Elasticity   = 0.0
# ...cut-off and opening lengths of bonds for T1
VertexModelSimulator.params.bond_T1_cutoff   = 0.04
VertexModelSimulator.params.bond_T1_opening   = 0.045

# ...vertex friction coefficient with external environment
VertexModelSimulator.params.xi       = 1.0

# ================== initial patterns of cell polarity (default=randomized) =============
VertexModelSimulator.params.P0_axissymmetric       = 1
# ...if P0_axissymmetric is set to 1, the code uses the parameters:
VertexModelSimulator.params.P0_a0_by_PI       = 0
VertexModelSimulator.params.P0_b0_by_PI       = 0

# ================== changes initial cellular network, for ensemble averages =============
VertexModelSimulator.params.run_id       = 1

# ...if you do not want to initialize with a predefined surface, set the following to zero
# ... if set to zero, a random tissue will be generated (default=1)
VertexModelSimulator.params.read_initsurface       = 1

#  ============================  parameters for active terms =============================
# ...cell traction force magnitude
VertexModelSimulator.params.F_mag   = 0.02
# ...rate of cell polarity alignment with neighbors
VertexModelSimulator.params.P_gamma   = 0.005
# ...rate of cell polarity alignment with cell velocity
VertexModelSimulator.params.P_nu   = 0.0
# ...strength of rotational noise in the polarity
VertexModelSimulator.params.P_noise   = 0.001
# ...polarity norm constrains
VertexModelSimulator.params.elastic_polarity       = 1
# ...if elastic_polarity is set to 1, instead of a hard constrain |p|=1, the molecular field
# ...in polarity p dynamics will include the term:  P_A * (1. |p|^2 ) * p
VertexModelSimulator.params.P_A       = 0.001

#  ======================  setting seed and initial noise used for debugging =============
VertexModelSimulator.params.noise   = 0.01
VertexModelSimulator.params.seed   = 1

In [13]:
VertexModelSimulator.RunSimulation()

Preparation run time :                    0.028 s
Algorithm run time :                    0.348 s


In [14]:
# ================================ initial configuration =====================
# ...to stop outputing results un-comment this:
#no_write       = 1
# ...should I write the data to vtk files?
VertexModelSimulator.analyzeParams.write_to_vtk       = 1
# ...should I analyze the cell elongation patterns?
VertexModelSimulator.analyzeParams.analyze_elongation       = 1
# ...should I decompose apical surface to vector spherical harmonics modes
VertexModelSimulator.analyzeParams.analyze_apical_surface_VSH       = 1
# ...should I decompose cell polarity field to vector spherical harmonics modes
VertexModelSimulator.analyzeParams.analyze_cell_polarity_VSH       = 1
# ...should I analyze the coarse-grained curvature tensor on defined patches
VertexModelSimulator.analyzeParams.compute_curvature_tensor       = 1
# ...should I analyze tissue rotation, angular velocity and residual from solid body?
VertexModelSimulator.analyzeParams.analyze_rotation       = 1
# ...should I align such that rotation axis points to z-direction?
VertexModelSimulator.analyzeParams.align_before_writing       = 1
# ...should I analyze data in co-rotating fram?
VertexModelSimulator.analyzeParams.analysis_in_corotating_frame       = 1
# ... what kind of data should be written to vtk?
VertexModelSimulator.analyzeParams.write_apical_polygonal_surface       = 1
VertexModelSimulator.analyzeParams.write_basal_polygonal_surface       = 0
VertexModelSimulator.analyzeParams.write_apical_triangulated_surface       = 1
VertexModelSimulator.analyzeParams.write_basal_triangulated_surface       = 1
VertexModelSimulator.analyzeParams.write_full_3D_triangulated_surface       = 1
VertexModelSimulator.analyzeParams.write_polarity_field       = 1
VertexModelSimulator.analyzeParams.write_nematic_field       = 1

# ==============================  Model specific parameters     ===================================
# ...the maximum l mode for vector spherical harmonics mode decomposition?
VertexModelSimulator.analyzeParams.Lmax       = 4
# ... first frame number inside data_dir to be analyzed
VertexModelSimulator.analyzeParams.first_frame       = 1
# ... last frame number inside data_dir to be analyzed
VertexModelSimulator.analyzeParams.last_frame       = 100

In [15]:
VertexModelSimulator.AnalyzeData()

100
Analysis run time :                    17.234 s


In [16]:
Plot=VertexModelSimulator.VizualizeIteration(10,edges=True)

Widget(value='<iframe src="http://localhost:46729/index.html?ui=P_0x7bd8c6110f80_1&reconnect=auto" class="pyvi…

In [17]:
VertexModelSimulator.VizualizeAnimate(gif_name="animation.gif",edges=True)

In [18]:
from IPython.display import HTML
HTML('<img src="./animation.gif">')